In [10]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Initialize Selenium WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Define the URL and tickers
yahoo_url = 'https://finance.yahoo.com/quote/'
tickers = ['AMZN','TSLA','META','MSFT','GOOG','NVDA','AAPL']

news_tables = {}
for ticker in tickers:
    url = yahoo_url + ticker + '/news'
    driver.get(url)
    
    # Scroll down to load all dynamic content
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Adjust timing based on your connection speed
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Use BeautifulSoup to parse the loaded page
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    news_table = soup.find('ul', class_='stream-items x-large layoutCol1 svelte-1siuiba')
    news_tables[ticker] = news_table

#     print(news_table)
    print(len(news_table.findAll('li'))) if news_table else print('No list items found')

driver.quit()



238
237
243
238
239
237
239


In [15]:
print(news_tables['MSFT'])

<ul class="stream-items x-large layoutCol1 svelte-1siuiba"><li class="stream-item svelte-7rcxn"><section class="container sz-x-large stream svelte-13zydns responsive" data-testid="storyitem"><a aria-label="Tesla, Apple, Nvidia: Factors keeping these stocks popular" class="subtle-link fin-size-small thumb svelte-wdkn18" data-rapid_p="19" data-v9y="0" data-ylk="elm:img;elmt:link;itc:0;ct:video;g:3eb3ea4f-a28a-3c98-b645-2be5d58d0f55;slk:Tesla%2C%20Apple%2C%20Nvidia%3A%20Factors%20keeping%20these%20stocks%20popular;sec:qsp-news;subsec:all;cpos:1" href="https://finance.yahoo.com/video/tesla-apple-nvidia-factors-keeping-153851961.html" title="Tesla, Apple, Nvidia: Factors keeping these stocks popular"><img alt="Tesla, Apple, Nvidia: Factors keeping these stocks popular" class="ar-small svelte-13zydns" loading="lazy" src="https://s.yimg.com/uu/api/res/1.2/4hp79zmAxCkN09yw7s06lQ--~B/Zmk9c3RyaW07aD0xMjY7cT04MDt3PTE2ODthcHBpZD15dGFjaHlvbg--/https://s.yimg.com/os/creatr-uploaded-images/2024-04/d8

In [18]:
print(news_tables.items())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
parsed_data = []

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)


for ticker, news_table in news_tables.items():
    print(ticker)
    i = 0  # Initialize the counter outside of the inner loop
    for elem in news_table.findAll('li'):
        print(i)
        i += 1  # Increment the counter here
        if i > 50:
            break  # Break the inner loop if more than 50 links have been processed

        try:
            atag = elem.find('a', class_='subtle-link fin-size-small titles noUnderline svelte-wdkn18')
            articleInfo = ''
            if atag:
                h3tag = atag.find('h3')
                if not h3tag:
                    continue  # If there's no h3 tag, move to the next item

                link = atag.get('href')
                newUrl = 'https://finance.yahoo.com' + link if not link.startswith('http') else link

                # Use Selenium to navigate to the new URL
                driver.get(newUrl)
                print(newUrl)
                html = BeautifulSoup(driver.page_source, 'html.parser')

                content = html.find('div', class_='caas-body')
                if content:
                    pTags = content.findAll('p')
                    for pTag in pTags:
                        if all(isinstance(child, NavigableString) for child in pTag.children):
                            articleInfo += pTag.text.strip() + ' '

                title = h3tag.text
                parsed_data.append([ticker, title, link, articleInfo])
        except WebDriverException as e:
            print(f"An error occurred while fetching or parsing the article: {e}")
            continue  # Skip to the next loop iteration if an error occurs

df = pd.DataFrame(parsed_data, columns=['ticker', 'title', 'link', 'articleInfo'])
print(df.head())
driver.quit()


AMZN
0
https://finance.yahoo.com/news/looking-slam-dunk-investment-put-160700853.html
1
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAIJbfYLMqkRDL9p9nwOH8i42MapRwltN7XBG8-NfA02i5HbDknv_zMb04LEMIsWcKHKHTLfYfrpJhPi_2cchDPChkwNM7FJZVjilVf9ZDi6m45neYp88xJtLxuSwvKCow_YTNa4EKpFcece117qWNIGYYoS3ZfRtW2qaeAL7T6tv2UCyCekLQQbNopoAJzPkOpC0Jjpwf7SoHQJNj035kLBzdOiJ2ZGzTlwXKd_KNQOAgDVK_1LY5HKi2GVnB3NQTv2KuG5AxqKa8dwhZ8uan4C31mDED46r5J9r2yrdqzdhqjsg-SR2nBmgsHFcSqLb0t24xW7aUtT9KU4CPWMLGJqDcuWldk4aM7LEwmySgFMxxHKk_pHpKQr1LKLSM_GsTUF94u9vJb_jxwvk3QlM84IpKp8W1ivpDwWpO_BAtUT8sYbDHdi72Sne-gajJ91wABMeKluke01UEtiCrQRXWQfOcdSqbklJN903tcbqFE8v5Q27hWwXann9PklmTyhPXlykWyFLOi_60hNuAdcunh3wwADAUpvJIu9xe0bxvJA5ARFQBmK68ZohlxljjMnuHu2pRHF8SCGOxlHQFIwb0fQFHAvUG7dVn15Ya3VD946vNHy6naYzABha_AlllmRIwzn86qdjpUq3eLUwuBNY3Hp94T0D9MwsOtqXalyOVb28x_64uqaw-uQRDrEOh3-LF7tKbrXNx21ou7mORhFJRcSkBGhdWuXGyp7c490eWd8wqPJbDiQo-vM5Zoil0PXGIEppRnKR5sj_s_NXLN71fScPxEohaxtbyb7-zFlHdDivbbZWaDAB4xzoaZ59vdLO0sCQwyCWkp1PdKIaSB2opyA_JM

https://finance.yahoo.com/news/magnificent-seven-stock-just-hit-095000304.html
13
https://finance.yahoo.com/news/two-years-historic-win-divided-092613860.html
14
https://finance.yahoo.com/news/best-stock-buy-now-shopify-090300210.html
15
https://finance.yahoo.com/news/3-top-tech-stocks-buy-224600585.html
16
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAFKSmZ5e-ZU0071MUgH-AXbvWc-AVgYVtnxYOw0FxFsiH1DY2GlJHoKy66SVaklSLejSkzxyAwkDBIwfZidnpmGIpEOMpt_rfwXel4tNXhwllPa445Dy9Ngyny8cGUdtwSwR20_rDP6mzLTP2r5OJrZ06EgNtKOOjBYD6HG8QhpcD2yvW0aNEtshJJaoJuSYVemLsZObSgx59sg7-Znzcma6oLYPHFSJeBdq2pG7f-1Wywagnp9jh8Mwtspnxu9RpwdUW9RZJiWAW5OsglG2bOofLNH4M9CFisUYmSK7STAoqKNooAikHCQQ1Lw1nvgXE_XDN8quBNGST3GjCB9FfhtbFwyVkajEJ7iv9zBiCtYw0GinI8lEVPRCeXKLEgAzWeh7L9GVOOgWuuHR1HgaBCJQkTWDT0XigCCkx_WU0ToFYMtjxn_N532cA3AHe5EXjDftyxIEFL8IwhREbWcRr0y_ZgMZK9pU_l19bOMLqDVB6zkJbzBOOWKPhtfaPmZJMrEklD2QQmo1UUmy7O3yIQd2RIanSRNgQM5uA7iDy6SXqIbUeBPcRJHTNeQCkD5cVzvXj9_D21vE5_efig2XEYMwZpbojuLCtIABNpKSWobqQotOsmPaXwcZ39QpF

https://finance.yahoo.com/news/biden-manufacturing-industry-expanding-could-110026813.html
28
https://finance.yahoo.com/news/2-trillion-dollar-magnificent-seven-102900001.html
29
https://finance.yahoo.com/news/forget-nvidia-3-magnificent-seven-094500601.html
30
https://finance.yahoo.com/m/e136a1c7-9425-3be9-a282-5c8293da5e83/big-tech-has-a-big-cash.html
31
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAGxDy_aQ7vVN_1ojAS6UV5hBOCt0KJtuAse8vT_2pOI-sxcMBK3tPaYQ17OIm83OcoU-BDVgyH08dkR0ig7Xb9gKoqhl6dHIjyuYTtWr-bMf1TwfQNViYcgirQfMVi9sSRlmReselaJIlpKYsErgW40hZTSqBA4yNm55wZRx4tfIgG_XsWCKkMRLh9GWf1_iUU9zM9vaFZYlpFb_NvE5AiNLT53sLbM--xj9DZo_5n8XRq9wducgzup5MyLEJUwaul82hsQQzu55h_BoHpUkrrggdAIbtff8pv3hhbFwVBMgCd4G_EWmMLJpClkBJ8yoVcCYBH0X2dcyaQ2iEHrVzcCj6UAHMtf0v0eYVKJfOykHSENtGHp-7gjVcNO4hNs4ZV1zSJA_ca1VeXF5GR_EryxzvCf8UDe09qVEPwXoB4Hk8jMAMO5uiClNLQ-lq5qC5zQDNywaa4PaGEm62aIG_aJvJjwLRlCKDGX0FWBet0NdAu7QvZKHjW6Q9Dyx7xIdfymz5YYrCjAFGy0m-WJCi--IxcRjswXb8Bh7Ne6xjGM4KGG8qBefAoRhY-ugdcW318AKHZcQzbv

https://finance.yahoo.com/news/history-says-nasdaq-soar-top-074500130.html
43
https://finance.yahoo.com/video/amazon-mgm-partners-youtuber-mrbeast-213526676.html
44
https://finance.yahoo.com/news/amazon-com-inc-amzn-q4-213310377.html
45
https://finance.yahoo.com/news/where-amazon-stock-1-210800798.html
46
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAANPEmkAf_SBk__NlIKJaca5vP__kVcFfCsyvJby-n98TMH9XKhqjQ4wzXV-Lsa_wKPfkJ9aKPJ_sGsnGxtRGEkqd_1TZTnCiifhkG9OB03Pz_sa2TRK-2-CO7aHlT5GcXra0jIc16hojsc-A-PjCbh3tuQxrFEE-rO-n3Jb8nscbupBshoUpAdVTdKQQyoMsKtG8diMjPRr6AyzCwT6vChlUgnP26oiTAFFOnRHRmVYUzgpMwXk7VbjGVmOGuSsEK1QztIH_KlSrt9_qAyXF5oINRx3yAINMN2M75OugtYqOcESWFuvNs7cvvrlZR-boE87G7GWrP6ci5MkHSNUIXFolHCN4uMy23nsSZycXySoHEdbIUyKJKBdgFVUYrcVqVFkV6_I9Y0x_xHrAKABJdoO1pjr8oSqZqmBk_CY3PaXxFxlJHMWDfzURocQa37UPMoedxo-DJpk8EZf7xb85wbM6YnguOq8wCJeTQ7uiEgfjQP4P9OkXM8imXnSauLkPSIembHWb54hE4I3Jl60dBJq9_nZy3QXCgpajzvDzjFElyG8wSKEuwuJ_lPaUISI8MA8O0h9HReTr1QvAGV-COIE-GxG5aABxxfvnL64cgPyQg2s2IOSG-fJAe7ppq1k

https://finance.yahoo.com/m/1900a22a-480d-302e-a601-f8ca56572645/these-stocks-are-moving-the.html
8
https://finance.yahoo.com/news/us-stocks-wall-st-gains-155600852.html
9
https://finance.yahoo.com/news/dogecoin-overtakes-pinterest-draftkings-market-150011560.html
10
https://finance.yahoo.com/m/d393b22e-22c0-3ca8-95fc-586ff9f2edbb/tesla-stock-rises-after.html
11
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAD21JO8rBvebgpD2MwrstbRAmvpIB3E8jC5Tsgnor0OVRyJ95J2gRATk-J6kquJLc5V9CYCTNT2kBLs1KZCx9YXyq790lzzT54IFd9sLMrEAenr_9CZXDiAQ4hxftg36OMgZqB4X0uS52dNkGif-6fs2wfDhglew5MKKR7ba6ENcvzIxKEXnytfyoSknBlpDWQT8QrgtJZ2RgNUWiPUbn3F17tYISFVtQSRl_88lpUULrimNgQLMZTfTq6zE2w1pjvQKeLGgmEqziBu4BhGbLHaSI5GMoxjkNgpjq_2qkkcDPD4RA7vfhEXCZhzIhqOfhB2jPEwHbrTrmqKuQCPjV2TLnmzeZrKpmiM5U4aCtUEpBaA1siIFOeA042Y2GqwHfQwr72T5tV_ikE3B8y8CBKwXA_04fOygwEXboPzz7uyeFHYqDRFrWIi73zf-_I6z9Qrx8F17nCMXKMk42RCnVjfehie75y4fVzLhTFcN-J_j2C2pHg_BwAAaa1eehdIUrFnrh8ImJ3Ncfl8k-uVbu2dUJv6fASm_sdgNfa4rTiNY7saIxPOkZl6A50KMgJNR0JNp2

...

https://finance.yahoo.com/news/nvidia-stock-buy-now-110500784.html
36
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAOiXCLSPMp-4B3osHrU3OyVg2Iuddj9aI5Tcn-ZbF3mGhcjO4fxEBRS9XRluXPw2o-sygjIp9pel7ZLKRpFy26EHXag4ejLi_NdAk6mqBb3oySQzb1bLEMR4S5peWgfBlbypNaZdm0k7lD7wZhT5IXk6nvikaSeNOTdjgHo7iyLY54g8cbyeEvTtGLlDx4jOcdrqN-CEhVhTxSj5syEfETEUxCxx5KO-n0KxcXWvQ0egk0HodwRic5qFnkQBpHx5SCLYlhPGToe6x5Njh798zivCCkHasNbA_0CHpN-aDE3QtPXclzrSvG5EW6M7-F0tVukYFvJFFOA_mFoOuG1RooHJnwaNPFiDi5GfY2WxUocE7w1DW6uPcj7uetnt-EpACSXbpAMTTV2caevl35I-bhZdyDvV3yoPvGyKdo9A5YzTe7bDyaSXc4GFls_UnQJrcM5tMjTXWHhBz-ktF5MsYu9gAdHC0YdxilCqYsNOKGWS0xnXGJPcqgtBBJMOueAjcQe9WClGwF5yuOzU43usGe2SqlzYY3AUMqTYPMwIALBPhljD-5U7rOh68qZzi1a9BhncCcv8uRztjTep1PMmRcX78JyX_f1COlr-axOKQVnZ2e7kGLIz8q6EQ5gFWGqZzG8dp-3IDdZgut7vFh-U3yr8njThl0YsjexHPksFsgmos_iD_JDly1j4Dl0efdn3SQiK__H3indB-R2-tZWcmF7cxKwvC6wZWV2bMhM5otQqitcctR5vmH-Y9AJbb4khKqQQ272YrGLszgBA5uCt9G3ESsadjNzbo0aTavZSU-p82wXUY9EErj3RGL3Pw3gmphyr63npNSWs_kF5aRwlxKYupEVvAgAz3k1NT1pSAgee

https://finance.yahoo.com/news/soaring-178-reality-may-setting-152800860.html
48
https://finance.yahoo.com/news/buy-sneaky-artificial-intelligence-stock-150000089.html
49
https://finance.yahoo.com/news/jim-cramer-bearish-10-stocks-142022171.html
50
AAPL
0
https://finance.yahoo.com/video/tesla-apple-nvidia-factors-keeping-153851961.html
1
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAE1WfzzWf24dlztz98C2Db2HbPFtmE7aYSIuc7nju66SEib8LCwBqNN4j8vsRB3cfeFNf_F4OwRKzfojmRK3xRxKAbV_5kz7GioLr-nWFCqunEAreOpWeUXPJPmImxsY6kkRNe1Im184HaXAnS-XMH6mkMqGi_XHuMBQD6COTX27bpROuankdThcQsgOCroIq_tpzhNAda_GYuZfud1CBip-bIqcp1Tb1hXNfnI-YIWfOocHrVHqeinLvL04vW6IwAdEG8WGPl80_XLARzbsYOSc0YitmYE8JTUUkDqaSD9WU5xxqaFwS8iwaiK7HS-x9tGVa8T_apUKS3WlUrnTixXq8ozyRYN6fld8ymL6QTTSxpN8xTdsi2UVIXJyfm6sVaRwkahw1PWjrOisnNRX-mtuLbNUoCV_HkFLes8Pz3y4zSDNOTAi306V0hFDUbXEnvOX7iEwYfBu7KlgjpNRlPqjsMFr25J8tarqG3c2ecUIK1rReQ1cFwH8CW1b0sCwaB7WDhFKmt_81_hvPmzgN6c4LMWZYmj8IuE3jqd5bRD8QrjUcp2N0_VZFlB0sfAIN-9nZdtDe8VD_H51-AVLsNso6iXccT

https://finance.yahoo.com/news/apple-suppliers-embark-groundbreaking-housing-094450990.html
13
https://finance.yahoo.com/news/almost-half-warren-buffetts-369-083500243.html
14
https://finance.yahoo.com/news/apples-success-teach-investors-pelotons-143000037.html
15
https://finance.yahoo.com/news/meet-best-performing-magnificent-seven-114500442.html
16
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAADDq-r4tbveInofJ5oZ2jY_UuknKZG4S14IIKzN-UwdpUlCIB0vDdPDBNG1wfF4MzhKCfEj029pWVSSeu1JBdDn7n4jmFHSEGyNTRkj_kd0ZDByOkD92GhLGueE4hamlPRh3bNQvHww04-EanYI7b6CJVHlvKD0gZ-1boqdV2FbfxDcyGiMY8R6nm945EV57DE_NCQLED5lqT-H9oGhAE00T7SN2HWSHQaHdCozp6X_bF_shQkPfG7RQt8M9rCnIjLFy9FtHUHHY0fdZowdYpcEcMuPEF0WJWm7s4rQ5LkXtndZJazS0rMzlKt4LsR_rK9JWkrVArQp6MP9xSDLLRrunfQmA2VbqjlET0HIUO_nNi1VnbVSNPq6MSkW7bEov1kLWfueeA_nr0akaOmSuQfiSXsBrfyviRWY2bVvaXnc0sQM-jqlDUDYb2UX_ItVQb_jUtG8TzQwYhA3s5CwMQpYgTye6kqP2knF3mugNpjP-D_bQqNIxJ-r-l-Mx5QTAeQniBh99zVILRQchgiY3yAbljj1Hw69006GkcwLXqLKJLoD32xvQEZKt4ABMG0oSQEjIk8MaDYeD-Aqvj

https://finance.yahoo.com/video/amazon-mgm-partners-youtuber-mrbeast-213526676.html
28
https://finance.yahoo.com/news/apple-inc-aapl-earnings-call-203740352.html
29
https://finance.yahoo.com/news/forget-dow-buy-magnificent-etf-201638625.html
30
https://finance.yahoo.com/news/apple-supplier-reports-revenue-dip-201444038.html
31
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAE7Vrjf5dtjb90Nm5nUx9RwtmptE3J5wyei02lr4F535ZKaa_tbqYDv-GK2ruLOQGZaV4T5gKWhg1LQkfk0fpCEu02gYPl-6b5eqtNhGWWMohk5C24chrOGfB2mBOnlkci6QN14HgSyYopdX2nf_vWRQInxGxJ0W9H5Xoj15X3rRrlgJyYn0LJNmXONPODlg49hm4VomI0s_TknT2ifZBSAkTih3fVAmFv1mjoSK7h8ycC8CtODPc71ptSCbd2F9U5OK44JfPwxEUkzLSYYsKKJu4hi0hbiDJs1S72qCd7jaV7nTXnpf_bwHvy8jrabhiiYV2ysqGT7IvW7UhcTkglHxM1Umenxw3bv3lurv42a-zrIWPcEWQrOdiObIEI_cIywx7PhR1exsLemhD5EAF-hMu4OEcBb6zwoOBfBbK7HueJOcv6BAlOf-7NdHPzaHZqcYOiXcrb3jSTNxnRnQkwxvwJarXAeScDBoDygdsrem0pENqfcbZCjmVDv1PjoHATOyYa-7YjkmnfaZRNdxNKIawPgQSoi7QDYmDTavx1yvMxMRfyl_wuq4IZnyjYXs3qBFIpTqJ5PQHfGA51iNyNKOMQ9LAQg_R6OruQQz4

https://finance.yahoo.com/news/insight-inside-big-techs-underground-100002984.html
43
https://finance.yahoo.com/news/forget-apple-warren-buffett-spent-090600538.html
44
https://finance.yahoo.com/news/hon-hai-sales-fall-short-081702325.html
45
https://finance.yahoo.com/news/1-apple-supplier-foxconn-sees-081413968.html
46
https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0&es=AQAAAMGkufcG2Q04eRyMEErsIoWoasgtRddVia58vNZ3aBauvvetiPAkD2t1jn3v1RvFTyFOxxM7hxnOFpdLuNAjEg4LsfUDDlo0NC2qztSk167OomRMpIAiTQPT2fpfYt5iSZPVraIRYXeyevIOmi7yhxRzZ0WoDjahLXIX7FwwCV3aYmmdiFmVJNMoSfxKGT-FGEqW3gXPKYA6L9psKMZ8pnqwkrorjIf25sPriUqPeoMJ37BDhvTGktWhNdho6UkG8YhJYOThUdK6icQAB1JIX2INIp6lyjTKBSmaGnp3BZO6A_2_Jr2SiDjBDGblRcVu0JxtAhvRtySbe5nEC8WAP1Tpj3lrmwDQvXRD6LovNaOzKmQeN2EmsZa7OzxZDp9FQ--Nx34ZFOSgR-Mq2S_NuBWhjTuK-9yg_qT_cSgdEWzLj5QpAxlvcddNRbZDUF-LZogI0WHZPg4sQF1-hdhTYgS5CwnLs7l65Zzb8tqSW9lAI3wPSrT0GEn_OPvY9eduQJMupN1cWM6sMNCEaveWIhG_ML1DRH6gPmJDJc2xZ_TiclayiOWQGoDAeQurSeiyHX_tEqATNx4BZdYCvWPNjBcECJfA17nJ6jXzrr1MqfLu-9MW

In [22]:
print(df)

    ticker                                              title  \
0     AMZN  Looking for a Slam-Dunk Investment? Put $1,000...   
1     AMZN           Switch and get a 5-year price guarantee.   
2     AMZN                 3 Biggest Value Investing Mistakes   
3     AMZN  Amazon's AI Eyesore. After Just Walk Out Fail,...   
4     AMZN  Amazon.com, Inc. (NASDAQ:AMZN) is favoured by ...   
..     ...                                                ...   
345   AAPL  UPDATE 1-Apple supplier Foxconn sees growth in...   
346   AAPL         Is it Time to Get Out of the Stock Market?   
347   AAPL  These 2 "Magnificent Seven" Stocks Are in the ...   
348   AAPL  Apple Lays Off 614 Workers After Canceling Car...   
349   AAPL  Apple supplier Foxconn sees growth in Q2 after...   

                                                  link  \
0    https://finance.yahoo.com/news/looking-slam-du...   
1    https://a.beap.gemini.yahoo.com/mbclk?bv=3.0.0...   
2    https://finance.yahoo.com/news/3-biggest

In [23]:
df.to_csv('stockNews.csv')